# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
#nltk.download()
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import re
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import multioutput
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
nltk.download('stopwords')
#from custom_transformer import StartingVerbExtractor
from sklearn.metrics import fbeta_score, make_scorer
import pickle

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('DisasterResponse', engine)



#category_names = df.columns.tolist()
df.drop('child_alone', axis = 1, inplace = True)

In [4]:
df['related'] = df.related.map(lambda x:1 if x == 2 else x)
df.groupby('related').count()



,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6116,6116,3390,6116,6116,6116,6116,6116,6116,6116,...,6116,6116,6116,6116,6116,6116,6116,6116,6116,6116
1,20064,20064,6763,20064,20064,20064,20064,20064,20064,20064,...,20064,20064,20064,20064,20064,20064,20064,20064,20064,20064


In [5]:
#id = df['id']
#df.drop('id', axis = 1, inplace = True)
X = df['message']
Y = df.iloc[:,4:]
category_names = Y.columns.tolist()


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    #Normalize Text:
    norm_word = text.lower()
    
    #Tokenize words:
    words = word_tokenize(norm_word)
    
    #Remove stop words:
    words = [w for w in words if w not in stopwords.words("english")]
    
    #Lemmatize word
    clean_tokens = [WordNetLemmatizer().lemmatize(w, pos = "v") for w in words]
    
    return clean_tokens

tokenize(X.iloc[0])

['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier


pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(RandomForestClassifier()))
                        ])

parameters = {'clf__estimator__n_estimators': [50, 100],
                  'clf__estimator__min_samples_split': [2, 3, 4],
                  'clf__estimator__criterion': ['entropy', 'gini']
                 }
cv = GridSearchCV(pipeline, param_grid=parameters)



    





### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

pipeline_fitted = pipeline.fit(X_train, y_train)


In [13]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)



### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
print(classification_report(y_test.values, y_prediction_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      3987
               request       0.81      0.37      0.50       925
                 offer       0.00      0.00      0.00        28
           aid_related       0.74      0.56      0.64      2207
          medical_help       0.59      0.09      0.15       428
      medical_products       0.70      0.07      0.13       261
     search_and_rescue       0.86      0.04      0.08       148
              security       0.00      0.00      0.00       103
              military       0.64      0.05      0.09       192
                 water       0.85      0.22      0.35       356
                  food       0.85      0.49      0.62       583
               shelter       0.84      0.26      0.40       466
              clothing       0.64      0.09      0.16        74
                 money       0.83      0.04      0.08       122
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, fbeta_score, make_scorer

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

parameters = { 'vect__max_df': (0.75, 1.0),
             #  'vect__stop_words': ('english', None),
                'clf__estimator__n_estimators': [10, 20, 50, 100],
                'clf__estimator__min_samples_split': [2, 3, 4, 5],
              'clf__estimator__criterion': ['entropy', 'gini']
              }

scorer = make_scorer(fbeta_score, beta = 2)
cv = GridSearchCV (pipeline, param_grid= parameters, scoring = 'f1_micro', n_jobs = -1)
cv.fit(X_train, y_train)


In [ ]:
y_prediction_test_cv = cv.predict(X_test)
print(classification_report(y_test.values, y_prediction_test_cv, target_names=Y.columns.values))

In [15]:
# Build a custom transformer which will extract the starting verb of a sentence
from sklearn.base import BaseEstimator,TransformerMixin
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [16]:
#Building Second Pipeline to Improve
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


pipeline2_fitted = pipeline2.fit(X_train, y_train)


IndexError: list index out of range

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_prediction_test_pip2 = pipeline2.predict(y_test)
print(classification_report(y_test.values, y_prediction_test_pip2, target_names=Y.columns.values))

### 9. Export your model as a pickle file

In [ ]:
filename = 'classifier.sav'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.